In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split

In [2]:
data = pd.read_csv('./feature_store/polynomial-features.csv')

In [4]:
features = data.drop(columns=['average_adg']).values
target = data['average_adg'].values

In [5]:
# konsersi ke tensor
features_tensor = torch.tensor(features, dtype=torch.float32)
target_tensor   = torch.tensor(target, dtype=torch.float32).unsqueeze(1)

In [6]:
# 2. Preprocessing Data
dataset = TensorDataset(features_tensor, target_tensor)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)



In [11]:
dataset

In [7]:
# 3. Definisikan Model MLP
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

input_size = features.shape[1]  # Jumlah fitur
hidden_size = 64  # Ukuran layer tersembunyi
output_size = 1  # Output tunggal (num_of_harvest)

model = MLP(input_size, hidden_size, output_size)


In [8]:
# 4. Definisikan Loss Function dan Optimizer
criterion = nn.MSELoss()  # Menggunakan MSE Loss untuk regresi
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [9]:
# 5. Latih Model
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [10/100], Loss: nan
Epoch [20/100], Loss: nan
Epoch [30/100], Loss: nan
Epoch [40/100], Loss: nan
Epoch [50/100], Loss: nan
Epoch [60/100], Loss: nan
Epoch [70/100], Loss: nan
Epoch [80/100], Loss: nan
Epoch [90/100], Loss: nan
Epoch [100/100], Loss: nan


In [14]:
# Saving model
import datetime

timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
model_save_path = f"./mlruns/models/model-{timestamp}.pth"

# Simpan model ke file dengan nama timestamped
torch.save(model.state_dict(), model_save_path)
print(f'Model saved to {model_save_path}')


Model saved to ./mlruns/models/model-20240604-185503.pth


In [10]:
# 6. Evaluasi Model
model.eval()
with torch.no_grad():
    test_loss = 0
    for batch_X, batch_y in test_loader:
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        test_loss += loss.item()
    
    avg_test_loss = test_loss / len(test_loader)
    print(f'Average Test Loss: {avg_test_loss:.4f}')


Average Test Loss: nan
